IMPORTS

In [77]:
!pip install faker
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import timedelta, datetime
faker = Faker()
random.seed(42)
np.random.seed(42)

DIMENSION TABLES

In [79]:
# dim_date
date_range = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D')
dim_date = pd.DataFrame({
    'date_key': date_range.strftime('%Y%m%d').astype(int),
    'full_date': date_range,
    'day': date_range.day,
    'month': date_range.month,
    'quarter': date_range.quarter,
    'year': date_range.year,
    'weekday': date_range.day_name()
})

In [80]:
# dim_plant
plant_ids = list(range(1, 11))
dim_plant = pd.DataFrame({
    'plant_id': plant_ids,
    'plant_name': [f'Plant {i}' for i in plant_ids],
    'region': random.choices(['USA', 'Germany', 'India', 'Mexico', 'China'], k=10)
})
print(dim_plant.head());
print(dim_plant.describe());

   plant_id plant_name   region
0         1    Plant 1   Mexico
1         2    Plant 2      USA
2         3    Plant 3  Germany
3         4    Plant 4  Germany
4         5    Plant 5   Mexico
       plant_id
count  10.00000
mean    5.50000
std     3.02765
min     1.00000
25%     3.25000
50%     5.50000
75%     7.75000
max    10.00000


In [81]:
# dim_shift
dim_shift = pd.DataFrame({
    'shift_id': [1, 2, 3],
    'shift_name': ['Morning', 'Afternoon', 'Night']
})
print(dim_shift.head());
print(dim_shift.describe());

   shift_id shift_name
0         1    Morning
1         2  Afternoon
2         3      Night
       shift_id
count       3.0
mean        2.0
std         1.0
min         1.0
25%         1.5
50%         2.0
75%         2.5
max         3.0


In [82]:
# dim_fault_type
fault_types = ['Overheat', 'Power Loss', 'Sensor Failure', 'Mechanical Jam', 'Oil Leakage']
dim_fault_type = pd.DataFrame({
    'fault_id': list(range(1, len(fault_types)+1)),
    'fault_name': fault_types
})
print(dim_fault_type.head());
print(dim_fault_type.describe());

   fault_id      fault_name
0         1        Overheat
1         2      Power Loss
2         3  Sensor Failure
3         4  Mechanical Jam
4         5     Oil Leakage
       fault_id
count  5.000000
mean   3.000000
std    1.581139
min    1.000000
25%    2.000000
50%    3.000000
75%    4.000000
max    5.000000


In [83]:
# dim_product
dim_product = pd.DataFrame({
    'product_id': range(1, 501),
    'product_name': [faker.word().capitalize() + " Part" for _ in range(500)],
    'category': random.choices(['Electrical', 'Mechanical', 'Chemical'], k=500),
    'unit': random.choices(['kg', 'liters', 'units'], k=500)
})
print(dim_product.head());
print(dim_product.describe());

   product_id   product_name    category    unit
0           1  Identify Part  Electrical      kg
1           2     Green Part  Mechanical      kg
2           3   Surface Part  Electrical   units
3           4       Eye Part  Electrical  liters
4           5      Deal Part  Mechanical   units
       product_id
count  500.000000
mean   250.500000
std    144.481833
min      1.000000
25%    125.750000
50%    250.500000
75%    375.250000
max    500.000000


In [84]:
# dim_operator
dim_operator = pd.DataFrame({
    'operator_id': range(1, 3001),
    'name': [faker.name() for _ in range(3000)],
    'experience_years': np.random.randint(1, 21, size=3000),
    'department': random.choices(['Assembly', 'Packaging', 'Welding', 'QA'], k=3000)
})
print(dim_operator.head());
print(dim_operator.describe());

   operator_id             name  experience_years department
0            1   Michelle Jones                 7    Welding
1            2     Ashley Moore                20    Welding
2            3  Kimberly Barnes                15    Welding
3            4    Rebecca Smith                11   Assembly
4            5      Jamie Patel                 8         QA
       operator_id  experience_years
count  3000.000000       3000.000000
mean   1500.500000         10.354000
std     866.169729          5.825804
min       1.000000          1.000000
25%     750.750000          5.000000
50%    1500.500000         10.000000
75%    2250.250000         15.000000
max    3000.000000         20.000000


In [85]:
from datetime import datetime

start_date = datetime.strptime('2010-01-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2020-01-01', '%Y-%m-%d').date()

# dim_machine
dim_machine = pd.DataFrame({
    'machine_id': range(1, 501),
    'machine_type': random.choices(['Drill', 'Lathe', 'Conveyor', 'Welder', 'Robot Arm'], k=500),
    'install_date': [faker.date_between(start_date=start_date, end_date=end_date) for _ in range(500)]
})
print(dim_machine.head());
print(dim_machine.describe());

   machine_id machine_type install_date
0           1        Drill   2012-10-03
1           2     Conveyor   2019-06-17
2           3        Lathe   2017-12-27
3           4    Robot Arm   2010-04-08
4           5       Welder   2013-02-07
       machine_id
count  500.000000
mean   250.500000
std    144.481833
min      1.000000
25%    125.750000
50%    250.500000
75%    375.250000
max    500.000000


FACT TABLES

In [87]:
NUM_ROWS_PRODUCTION = 200000
NUM_ROWS_QUALITY = 100000
NUM_ROWS_DOWNTIME = 80000
NUM_ROWS_COST = 150000
NUM_ROWS_INVENTORY = 200000

# Helper Lists
date_keys = dim_date['date_key'].tolist()
product_ids = dim_product['product_id'].tolist()
plant_ids = dim_plant['plant_id'].tolist()
operator_ids = dim_operator['operator_id'].tolist()
machine_ids = dim_machine['machine_id'].tolist()
fault_ids = dim_fault_type['fault_id'].tolist()
shift_ids = dim_shift['shift_id'].tolist()


In [88]:
# fact_production_orders
fact_production_orders = pd.DataFrame({
    'order_id': range(1, NUM_ROWS_PRODUCTION+1),
    'product_id': np.random.choice(product_ids, NUM_ROWS_PRODUCTION),
    'plant_id': np.random.choice(plant_ids, NUM_ROWS_PRODUCTION),
    'date_key': np.random.choice(date_keys, NUM_ROWS_PRODUCTION),
    'shift_id': np.random.choice(shift_ids, NUM_ROWS_PRODUCTION),
    'planned_qty': np.random.randint(100, 1000, NUM_ROWS_PRODUCTION)
})
fact_production_orders['actual_qty'] = fact_production_orders['planned_qty'] - np.random.randint(-50, 100, NUM_ROWS_PRODUCTION)
print(fact_production_orders.head());
print(fact_production_orders.describe());

   order_id  product_id  plant_id  date_key  shift_id  planned_qty  actual_qty
0         1         109         5  20241220         2          252         246
1         2         224         1  20231118         1          834         803
2         3         496         2  20220611         3          492         395
3         4         278         6  20221011         3          333         317
4         5          68         2  20240708         2          605         514
            order_id     product_id       plant_id      date_key  \
count  200000.000000  200000.000000  200000.000000  2.000000e+05   
mean   100000.500000     249.998370       5.491125  2.023064e+07   
std     57735.171256     144.265929       2.871282  8.174916e+03   
min         1.000000       1.000000       1.000000  2.022010e+07   
25%     50000.750000     125.000000       3.000000  2.022093e+07   
50%    100000.500000     250.000000       5.000000  2.023070e+07   
75%    150000.250000     375.000000       8.000000

In [89]:
# fact_quality_logs
fact_quality_logs = pd.DataFrame({
    'log_id': range(1, NUM_ROWS_QUALITY+1),
    'product_id': np.random.choice(product_ids, NUM_ROWS_QUALITY),
    'plant_id': np.random.choice(plant_ids, NUM_ROWS_QUALITY),
    'date_key': np.random.choice(date_keys, NUM_ROWS_QUALITY),
    'operator_id': np.random.choice(operator_ids, NUM_ROWS_QUALITY),
    'defect_type': np.random.choice(['Scratch', 'Dent', 'Misalignment', 'Color Deviation'], NUM_ROWS_QUALITY),
    'rework_hours': np.round(np.random.uniform(0.5, 5.0, NUM_ROWS_QUALITY), 2)
})
print(fact_quality_logs.head());
print(fact_quality_logs.describe());

   log_id  product_id  plant_id  date_key  operator_id   defect_type  \
0       1         456         2  20240423         1145       Scratch   
1       2          84         4  20230514          785          Dent   
2       3         372         9  20231107         2726  Misalignment   
3       4          98         6  20240520          780  Misalignment   
4       5         292         8  20240919         1278  Misalignment   

   rework_hours  
0          2.04  
1          4.25  
2          3.02  
3          1.85  
4          1.09  
              log_id     product_id       plant_id      date_key  \
count  100000.000000  100000.000000  100000.000000  1.000000e+05   
mean    50000.500000     249.768870       5.496480  2.023066e+07   
std     28867.657797     144.057041       2.876385  8.163361e+03   
min         1.000000       1.000000       1.000000  2.022010e+07   
25%     25000.750000     125.000000       3.000000  2.022100e+07   
50%     50000.500000     249.000000       6.000000 

In [90]:
# fact_machine_downtime
fact_machine_downtime = pd.DataFrame({
    'event_id': range(1, NUM_ROWS_DOWNTIME+1),
    'machine_id': np.random.choice(machine_ids, NUM_ROWS_DOWNTIME),
    'plant_id': np.random.choice(plant_ids, NUM_ROWS_DOWNTIME),
    'date_key': np.random.choice(date_keys, NUM_ROWS_DOWNTIME),
    'fault_id': np.random.choice(fault_ids, NUM_ROWS_DOWNTIME),
    'operator_id': np.random.choice(operator_ids, NUM_ROWS_DOWNTIME),
    'downtime_minutes': np.random.randint(10, 240, NUM_ROWS_DOWNTIME)
})
print(fact_machine_downtime.head());
print(fact_machine_downtime.describe());

   event_id  machine_id  plant_id  date_key  fault_id  operator_id  \
0         1         212         3  20220722         1         1965   
1         2         482         8  20241118         1         2660   
2         3           2         1  20230830         5          566   
3         4         284        10  20240528         2         1948   
4         5         366         6  20220311         3         2895   

   downtime_minutes  
0               181  
1                16  
2                40  
3               237  
4               196  
           event_id    machine_id      plant_id      date_key      fault_id  \
count  80000.000000  80000.000000  80000.000000  8.000000e+04  80000.000000   
mean   40000.500000    250.178712      5.497250  2.023068e+07      2.991788   
std    23094.155105    144.297703      2.863332  8.171362e+03      1.415299   
min        1.000000      1.000000      1.000000  2.022010e+07      1.000000   
25%    20000.750000    125.000000      3.000000  2.0

In [91]:
# fact_cost_journals
fact_cost_journals = pd.DataFrame({
    'entry_id': range(1, NUM_ROWS_COST+1),
    'product_id': np.random.choice(product_ids, NUM_ROWS_COST),
    'plant_id': np.random.choice(plant_ids, NUM_ROWS_COST),
    'date_key': np.random.choice(date_keys, NUM_ROWS_COST),
    'standard_cost': np.round(np.random.uniform(50, 500, NUM_ROWS_COST), 2)
})
fact_cost_journals['actual_cost'] = fact_cost_journals['standard_cost'] + np.round(np.random.uniform(-20, 50, NUM_ROWS_COST), 2)
print(fact_cost_journals.head());
print(fact_cost_journals.describe());

   entry_id  product_id  plant_id  date_key  standard_cost  actual_cost
0         1         136         2  20241021         279.25       326.54
1         2         412         6  20220107         402.08       399.36
2         3         378        10  20230910          54.57        48.24
3         4         453         4  20231125         139.38       172.66
4         5          50         4  20220605         479.79       515.45
            entry_id     product_id       plant_id      date_key  \
count  150000.000000  150000.000000  150000.000000  1.500000e+05   
mean    75000.500000     250.210427       5.499060  2.023070e+07   
std     43301.414527     144.486220       2.876092  8.184323e+03   
min         1.000000       1.000000       1.000000  2.022010e+07   
25%     37500.750000     125.000000       3.000000  2.022100e+07   
50%     75000.500000     250.000000       5.000000  2.023070e+07   
75%    112500.250000     376.000000       8.000000  2.024040e+07   
max    150000.000000    

In [92]:
# fact_inventory_movements
fact_inventory_movements = pd.DataFrame({
    'movement_id': range(1, NUM_ROWS_INVENTORY+1),
    'product_id': np.random.choice(product_ids, NUM_ROWS_INVENTORY),
    'plant_id': np.random.choice(plant_ids, NUM_ROWS_INVENTORY),
    'date_key': np.random.choice(date_keys, NUM_ROWS_INVENTORY),
    'movement_type': np.random.choice(['IN', 'OUT'], NUM_ROWS_INVENTORY),
    'quantity': np.random.randint(10, 500, NUM_ROWS_INVENTORY)
})
print(fact_inventory_movements.head());
print(fact_inventory_movements.describe());

   movement_id  product_id  plant_id  date_key movement_type  quantity
0            1         102         2  20241124            IN       443
1            2         131         1  20240914            IN       447
2            3          72         1  20230106            IN       458
3            4         153         5  20231227            IN        56
4            5          61         9  20220316           OUT       188
         movement_id     product_id       plant_id      date_key  \
count  200000.000000  200000.000000  200000.000000  2.000000e+05   
mean   100000.500000     250.712820       5.504680  2.023067e+07   
std     57735.171256     144.382982       2.871042  8.182531e+03   
min         1.000000       1.000000       1.000000  2.022010e+07   
25%     50000.750000     125.000000       3.000000  2.022093e+07   
50%    100000.500000     251.000000       6.000000  2.023070e+07   
75%    150000.250000     376.000000       8.000000  2.024040e+07   
max    200000.000000     500.0

SAVE AS CSV FILES

In [94]:
dim_date.to_csv('dim_date.csv', index=False)
dim_plant.to_csv('dim_plant.csv', index=False)
dim_shift.to_csv('dim_shift.csv', index=False)
dim_fault_type.to_csv('dim_fault_type.csv', index=False)
dim_product.to_csv('dim_product.csv', index=False)
dim_operator.to_csv('dim_operator.csv', index=False)
dim_machine.to_csv('dim_machine.csv', index=False)

fact_production_orders.to_csv('fact_production_orders.csv', index=False)
fact_quality_logs.to_csv('fact_quality_logs.csv', index=False)
fact_machine_downtime.to_csv('fact_machine_downtime.csv', index=False)
fact_cost_journals.to_csv('fact_cost_journals.csv', index=False)
fact_inventory_movements.to_csv('fact_inventory_movements.csv', index=False)
